<a href="https://colab.research.google.com/github/francescapoli98/pattern-recognition-systems/blob/main/midterm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intelligent Systems for Pattern Recognition: midterm 1**

#### Francesca Poli [560190]

##### **Assignment 3: signal processing**

The musical pitch of a note is determined by its fundamental frequency. The pitch played by different instruments sounds different due to harmonics, i.e. other frequencies that are superimposed and determine the timbre of the instrument. [This dataset](https://philharmonia.co.uk/resources/sound-samples/) contains samples from several instruments playing different notes. Plot the spectrogram for some of them (4 instruments are sufficient) and check if it is possible to recognize the different instruments by only looking at the spectrogram. In your presentation, discuss which samples you chose to compare, how you computed the spectrogram and whether the resulting features are sufficient to recognize the instrument.

In Python you can import WAVs (and acces several other music-related functions), using the [LibROSA](https://librosa.org/doc/latest/index.html) library.

##### Starting with LibROSA library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#some always-useful libraries
import numpy as np
import matplotlib.pyplot as plt

Let's get started with librosa!

In [3]:
%pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import librosa
print(librosa.__version__)

0.8.1


##### Loading the dataset with all instruments

In [5]:
import re
import os as os, fnmatch as f
import zipfile as z

instruments={}
suffix='.zip'

for dirname in os.listdir('/content/drive/MyDrive/ISPR/midterm1/all-samples'):
  #print('Instrument:',dirname)
  archive = z.ZipFile('/content/drive/MyDrive/ISPR/midterm1/all-samples/{}'.format(dirname),'r')
  files = archive.namelist()
  print('mp3 files in',dirname,':',files)
  instrument_name = dirname.removesuffix(suffix)
  instruments[instrument_name] = files
  archive.extractall(instrument_name)

mp3 files in bassoon.zip : ['bassoon_A2_025_forte_normal.mp3', 'bassoon_A2_025_fortissimo_normal.mp3', 'bassoon_A2_025_mezzo-piano_normal.mp3', 'bassoon_A2_025_piano_normal.mp3', 'bassoon_A2_05_forte_normal.mp3', 'bassoon_A2_05_fortissimo_normal.mp3', 'bassoon_A2_05_mezzo-forte_normal.mp3', 'bassoon_A2_05_mezzo-piano_normal.mp3', 'bassoon_A2_05_piano_normal.mp3', 'bassoon_A2_15_piano_normal.mp3', 'bassoon_A2_1_forte_normal.mp3', 'bassoon_A2_1_mezzo-forte_normal.mp3', 'bassoon_A2_1_mezzo-piano_normal.mp3', 'bassoon_A2_1_piano_normal.mp3', 'bassoon_A3_025_forte_normal.mp3', 'bassoon_A3_025_fortissimo_normal.mp3', 'bassoon_A3_025_mezzo-piano_normal.mp3', 'bassoon_A3_025_piano_normal.mp3', 'bassoon_A3_05_forte_normal.mp3', 'bassoon_A3_05_fortissimo_normal.mp3', 'bassoon_A3_05_mezzo-piano_normal.mp3', 'bassoon_A3_05_piano_normal.mp3', 'bassoon_A3_15_forte_normal.mp3', 'bassoon_A3_15_mezzo-piano_normal.mp3', 'bassoon_A3_15_piano_normal.mp3', 'bassoon_A3_1_forte_normal.mp3', 'bassoon_A3_1_for

In [6]:
#list of all the instruments at disposal
instruments.keys()

dict_keys(['bassoon', 'banjo', 'bass clarinet', 'cello', 'double bass', 'contrabassoon', 'clarinet', 'cor anglais', 'saxophone', 'mandolin', 'flute', 'guitar', 'oboe', 'french horn', 'percussion', 'tuba', 'trumpet', 'trombone', 'viola', 'violin'])

In [7]:
#check for dupes in the unzipped files! 
import collections 

seen = set()
for instrument in instruments:
  if os.listdir(instrument):
    if not f.fnmatch(instrument, '*.zip'):
      for file in os.listdir(instrument):
        if file not in seen:
          seen.add(file)
        else:
          print(file) 

I decide to allocate the sturments according to their musical family and I also choose four instruments to complete the initial task. 

Don't mind me if I go for a string quartet!

In [8]:
strings = ['banjo', 'violin', 'viola', 'cello', 'double bass', 'mandolin', 'guitar']
woodwinds = ['bassoon','bass clarinet', 'contrabassoon', 'clarinet', 'cor anglais', 'flute', 'oboe' ]
brass = ['saxophone', 'french horn', 'tuba', 'trumpet', 'trombone']
percussion =['percussion']

#instruments for the basic request of the midterm:

quartet = ['violin', 'viola', 'cello', 'double bass']

##### Plot the spectrogram

In [9]:
#function to plot the spectogram
import librosa.display

def spectrogram(S_db, plottext):
  fig, ax = plt.subplots()
  img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
  #ax.set(title='{}'.format(plottext))
  fig.colorbar(img, ax=ax, format="%+2.f dB")


Now I plot:
- the spectrogram for all the notes of every instrument, in order to have a nice general view 
- a spectrogram for every note throughout all the instruments to get the differences between each one of them and the pitches as requested.

In [ ]:
import pathlib
sounds=dict

for elem in quartet:  
  if os.listdir(elem):
    if not f.fnmatch(elem, '*.zip'):
      print('SPECTROGRAMS FOR INSTRUMENT:'+elem)
      for file in os.listdir(elem):
        path = str(pathlib.Path().resolve())
        #print(path)
        y, sr = librosa.load(str(path+'/'+elem+'/'+file))
        #print(y, sr)
        D = librosa.stft(y)  # STFT of y
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        text = str(file)
        spectrogram(S_db, text)

SPECTROGRAMS FOR INSTRUMENT:violin


<ipython-input-9-63e0b7182ee0>:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


SPECTROGRAMS FOR INSTRUMENT:viola


/usr/local/lib/python3.9/dist-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1728
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


EOFError: ignored